# "On your mark...get, set, GO"
> "Final config steps before we get the car moving..."

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [SelfDriving]
- hide: false
- search_exclude: false
- image: images/post-thumbnails/Final_Steps.png
- metadata_key1: MUSHR
- metadata_key2: 

# Purpose

Create a map, configure VISUALIZATION and watch the car self-drive!

![](https://abhisheksreesaila.github.io/blog/images/general/rviz_process_flowchart.png "Process Flow")


## Visualization

### What is RVIZ and how do i get it?

rviz is a 3D visualizer for the Robot Operating System (ROS) framework. Here we need to visualize and track car's movements in your laptop computer. So you have to set the variables correctly so that CAR can send the messages to your computer and vice-versa.

#### FAQs

**1. Why can't I open RVIZ directly by SSH-ing to car? In other words, why do i need laptop computer to open up RVIZ?**
RVIZ does not support X11 windows, and hence ssh -X will not work

**2. I have a mac/windows. How do I use RVIZ?**
Download the virtual machine image that already has the MuSHR stack and follow instructions
[here](https://mushr.io/tutorials/quickstart/) (highly recommended)


### Set the ROS IP's Variables

The setup is best shown visually below

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/rviz_setup.png "RVIZ Setup")

**Instructions**

1) Open **.bashrc** and set the variables. This will make sure we **dont have to do this everytime** you open a command shell. (trust me, you have to open a ton of times for testing)

```bash

sudo nano ~/.bashrc

```

Go to the last line of the script and add the following lines. Depending on whether you are setting the variables in the car or your laptop, set the **IP ADDRESS** accordingly. 

```bash

export ROS_IP= <<LAPTOP_IP_ADDRESS>>
export ROS_MASTER_URI=http://<<CAR_IP_ADDRESS>>:11311

```

For those wondering how to get the IP ADDRESS

```bash

# To get the ip address
ifconfig 

```

LAPTOP IP Address will be dynamic by default and might be pain to enter each time you login. You can use the following script to extract "IP Address" from the configuration.


![](https://abhisheksreesaila.github.io/blog/images/software-config/laptop-ip-address.png "laptop ip address")

**enp0s17 = wireless interface that shows the IP address**


laptop-ip-address


```
ifconfig enp0s17 | grep "inet" | awk 'NR==1{print $2}'

```



> Important: **NETWORK SETTING on the ORACLE VIRTUAL BOX**

Make sure the NETWORK SETTING on the ORACLE VIRTUAL BOX used to load MUSHR SIM (linux image) has "Bridge Adapter" (NAT is default. NAT will NOT work. It will translate the IP from the CAR to SIM and hence wont read any messages)

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/network_setting.png "Correct Network Setup")



## Maps

For any car to drive, we need maps so that car can understanding the boundaries and navigate accordingly.

### Create a map of your surrounding

#### Use GMAPPING Library

Start each one of following commands in a separate window as suggested [here](https://mushr.io/tutorials/navigation)

```bash

# Start teleop  (1st window)
roslaunch mushr_base teleop.launch

# Start gmapping scan (2nd window). MUSHR with a LIDAR publishes on the "base_scan" topic

rosrun gmapping slam_gmapping scan:=scan

```

Drive around the house or park or whereever. 

**Tips for producing a good map**

- Drive slowly. 
- Drive around the same place couple times
- Finish a loop


We can save the built map using the following command. This command will listen to the map topic and save into the image. The map server package does this operation

```bash

rosrun map_server map_saver -f real-floor0

```

The map generated will be clumsy at times.

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/my_map_untouched.png "Map Created By GMAPPING")

But, we can correct it using any photo editing software. After using [GIMP](https://www.gimp.org/) it was modified to

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/my_map_touched.png "Map After Editing")

> Note: You might to download the file to your local desktop, Open in GIMP/Photoshop to edit the software.

```bash

#download to local desktop
scp robot@<<CAR IP>>:~/real-floor0.pgm ~/Desktop/

```
```bash

#note that this uploads to home folder. Move it to appropriate folder later
#upload from local desktop to car
scp  ~/Desktop/real-floor0.pgm robot@<<CAR IP>>:~

```
 
 
### CROP Map

**By default the map contains an abundance of blank space around the area of detected surfaces** By default the file will be 16MB, which will slow down the HALTON SAMPLER when you load the maps and run your car. So we need to crop unnecessary part of the image. But care has to be taken not to *blindly crop* and lose the origin. For this purpose there is an undocumented utility [CROP_MAP]('https://github.com/abhisheksreesaila/blog/blob/master/assets/softwares/crop_map') to crop the map, removing the blank spaces around the map.In my case, it reduced by 20x. 

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/map_size_reduce.png "Map Dimensions Reduce")



### Edit CONFIG files to use the map

Navigate to MUSHR RHC maps directory and place your maps. Note that you need both the PGM and YAML file. 

> Note: Make sure the YAML file is referring the correct PGM file

```bash

roscd mushr_rhc_ros
cd maps

#copy pgm files to the current directory (mushr_base/maps)
cp "pgm files from whichever directory you have them" . 


```

Now, change the launch files

```bash

#go 1 level higher. Current directory assumed (mushr_base/maps)
cd ..

cd launch

#edit launch files. see screenshot
nano map_server.launch

```

![](https://abhisheksreesaila.github.io/blog/images/mushr-build-pics/map_server_launch_change.png "Map Server Launch File")


# References

[Connect to RVIZ from the linux image](http://www.youbot-store.com/wiki/index.php/Visualizing_performance_of_robot_connected_via_SSH_in_Rviz_on_master_device)

[Low Voltage Video](https://www.youtube.com/watch?v=clsBhxOo34Q)
